In [1]:
import numpy as np
import pandas as pd
import re
import numpy as np
import pandas as pd
import numpy as np
import string
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split

In [11]:
train = pd.read_csv("labeled.csv")
train.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [12]:
train.shape

(14412, 2)

In [13]:
def clean_text(text):
    
    text = text.lower()
    
    #pattern = [zero or more character]
    text = re.sub('\[.*?\]', '', text)
    
    #pattern = with or without(http),://, one or more non-white space character, OR www, .,one or more non-white space character
    text = re.sub('https?://\S+|www\.\S+', '', text)
    
    #pattern = <, zero or more characters, >, (one or more occurance of >)
    text = re.sub('<.*?>+', '', text)
    
    #pattern = any punctionation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    
    #pattern = any new line
    text = re.sub('\n', '', text)
    
    #pattern = any from[a-zA-Z0-9_], any from[0-9], any from [a-zA-Z0-9_]
    text = re.sub('\w*\d\w*', '', text)
    return text

In [14]:
train['clean_text'] = train['comment'].apply(str).apply(lambda x: clean_text(x))

In [15]:
train

,comment,toxic,clean_text
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0,верблюдовто за что дебилы бл
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0,хохлы это отдушина затюканого россиянина мол в...
2,Собаке - собачья смерть\n,1.0,собаке собачья смерть
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0,страницу обнови дебил это тоже не оскорбление ...
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,тебя не убедил пдф в том что скрипалей отрави...
...,...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0,вонючий совковый скот прибежал и ноет а вот и ...
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0,а кого любить гоблина тупорылого чтоли или как...
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0,посмотрел утомленных солнцем и оказалось что ...
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0,крымотред нарушает правила раздела тк в нем не...


## Tokenization

In [16]:
tokenization = nltk.tokenize.RegexpTokenizer(r'\w+')
train['clean_text_tk'] = train['clean_text'].apply(lambda x: tokenization.tokenize(x))
train

,comment,toxic,clean_text,clean_text_tk
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0,верблюдовто за что дебилы бл,"[верблюдовто, за, что, дебилы, бл]"
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0,хохлы это отдушина затюканого россиянина мол в...,"[хохлы, это, отдушина, затюканого, россиянина,..."
2,Собаке - собачья смерть\n,1.0,собаке собачья смерть,"[собаке, собачья, смерть]"
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0,страницу обнови дебил это тоже не оскорбление ...,"[страницу, обнови, дебил, это, тоже, не, оскор..."
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,тебя не убедил пдф в том что скрипалей отрави...,"[тебя, не, убедил, пдф, в, том, что, скрипалей..."
...,...,...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0,вонючий совковый скот прибежал и ноет а вот и ...,"[вонючий, совковый, скот, прибежал, и, ноет, а..."
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0,а кого любить гоблина тупорылого чтоли или как...,"[а, кого, любить, гоблина, тупорылого, чтоли, ..."
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0,посмотрел утомленных солнцем и оказалось что ...,"[посмотрел, утомленных, солнцем, и, оказалось,..."
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0,крымотред нарушает правила раздела тк в нем не...,"[крымотред, нарушает, правила, раздела, тк, в,..."


## Stop words

In [17]:
stop_words = set(stopwords.words('russian'))
def removing_stopwords(token):
    return [w for w in token if not w.lower() in stop_words]

In [19]:
train['clean_text_st'] = train['clean_text_tk'].apply(lambda x : removing_stopwords(x))
train.head()

,comment,toxic,clean_text,clean_text_tk,clean_text_st
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0,верблюдовто за что дебилы бл,"[верблюдовто, за, что, дебилы, бл]","[верблюдовто, дебилы, бл]"
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0,хохлы это отдушина затюканого россиянина мол в...,"[хохлы, это, отдушина, затюканого, россиянина,...","[хохлы, это, отдушина, затюканого, россиянина,..."
2,Собаке - собачья смерть\n,1.0,собаке собачья смерть,"[собаке, собачья, смерть]","[собаке, собачья, смерть]"
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0,страницу обнови дебил это тоже не оскорбление ...,"[страницу, обнови, дебил, это, тоже, не, оскор...","[страницу, обнови, дебил, это, оскорбление, до..."
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,тебя не убедил пдф в том что скрипалей отрави...,"[тебя, не, убедил, пдф, в, том, что, скрипалей...","[убедил, пдф, скрипалей, отравила, россия, ана..."


## Lematization

In [20]:
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatize_word(tokenization):
    return [wordnet_lemmatizer.lemmatize(token, pos="v") for token in tokenization]

In [21]:
train['Text_Lemmatization'] = train['clean_text_st'].apply(lambda x : lemmatize_word(x))
train['Text_Lemmatization']

0                                [верблюдовто, дебилы, бл]
1        [хохлы, это, отдушина, затюканого, россиянина,...
2                                [собаке, собачья, смерть]
3        [страницу, обнови, дебил, это, оскорбление, до...
4        [убедил, пдф, скрипалей, отравила, россия, ана...
                               ...                        
14407    [вонючий, совковый, скот, прибежал, ноет, стор...
14408    [кого, любить, гоблина, тупорылого, чтоли, как...
14409    [посмотрел, утомленных, солнцем, оказалось, эт...
14410    [крымотред, нарушает, правила, раздела, тк, не...
14411    [сих, пор, пересматриваю, видео, орамбо, кстат...
Name: Text_Lemmatization, Length: 14412, dtype: object

In [22]:
def combine_text(list_of_text):
    '''Takes a list of text and combines them into one large chunk of text.'''
    combined_text = ' '.join(list_of_text)
    return combined_text

train['Final_text'] = train['Text_Lemmatization'].apply(lambda x : combine_text(x))
train['Final_text']

0                                    верблюдовто дебилы бл
1        хохлы это отдушина затюканого россиянина мол в...
2                                    собаке собачья смерть
3        страницу обнови дебил это оскорбление доказанн...
4        убедил пдф скрипалей отравила россия анализиро...
                               ...                        
14407    вонючий совковый скот прибежал ноет сторонник ...
14408    кого любить гоблина тупорылого чтоли какуюнибу...
14409    посмотрел утомленных солнцем оказалось это хор...
14410    крымотред нарушает правила раздела тк нем обсу...
14411    сих пор пересматриваю видео орамбо кстати свое...
Name: Final_text, Length: 14412, dtype: object

In [23]:
from sklearn.pipeline import Pipeline

In [24]:
x_train,x_test,y_train,y_test = train_test_split(train['Final_text'],train['toxic'],test_size=0.2,random_state=42)

In [25]:
pipe_lr = Pipeline(steps=[('cv',TfidfVectorizer()),('lr',LogisticRegression())])

In [26]:
pipe_lr.fit(x_train,y_train)

Pipeline(steps=[('cv', TfidfVectorizer()), ('lr', LogisticRegression())])

In [27]:
pipe_lr.score(x_test,y_test)

0.8192854665279223

In [28]:
pipe_lr_1 = Pipeline(steps=[('cv',CountVectorizer()),('lr',LogisticRegression())])

In [29]:
pipe_lr_1.fit(x_train,y_train)

Pipeline(steps=[('cv', CountVectorizer()), ('lr', LogisticRegression())])

In [30]:
pipe_lr_1.score(x_test,y_test)

0.8446063128685397

In [31]:
from sklearn.ensemble import RandomForestClassifier

In [32]:
pipe_lr_2 = Pipeline(steps=[('cv',CountVectorizer()),('lr',RandomForestClassifier())])

In [33]:
pipe_lr_2.fit(x_train,y_train)

Pipeline(steps=[('cv', CountVectorizer()), ('lr', RandomForestClassifier())])

In [34]:
pipe_lr_2.score(x_test,y_test)

0.8015955601803677

In [36]:
from sklearn.svm import SVC

In [37]:
pipe_lr_3 = Pipeline(steps=[('cv',CountVectorizer()),('lr',SVC())])

In [38]:
pipe_lr_3.fit(x_train,y_train)

Pipeline(steps=[('cv', CountVectorizer()), ('lr', SVC())])

In [39]:
pipe_lr_3.score(x_test,y_test)

0.7991675338189386

In [35]:
import joblib
pipeline_file = open("RU_toxic_classifier_model.pkl","wb")
joblib.dump(pipe_lr_1,pipeline_file)
pipeline_file.close()